<a href="https://colab.research.google.com/github/tarunku/open_llm/blob/main/102_ReActAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain_community langchainhub langchain-openai langgraph

In [1]:
import os
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = userdata.get("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = userdata.get("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [2]:
llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))


<ipython-input-2-7167b0a4fcd9>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))


In [3]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor
)

from langchain import hub
import inflect
import re

In [4]:
@tool(name_or_callable='next_num_tool')
def next_num(a) -> str:
    """get the next number."""
    return inflect.engine().number_to_words(int(a) + 12)



In [5]:
def agent_007(user_query: str):
  template = '''
    You are a smart math assistant designed to compute the next number based on a given input.

    You have access to the following tool:

    **next_num(a: int):** Returns the next number by adding 12 to the given input and converting it to words.

    When given a user query, determine whether it requires this tool. If it does, use the tool to perform the calculation and return only the final result.

    If the available tool is not sufficient to answer the query, respond with: "I can't handle this query."

    -----------
    Now here is the user query: {user_query}
    '''

  prompt_template = PromptTemplate(
        template=template, input_variables=["user_query"]
    )

  react_prompt = hub.pull("hwchase17/react")

  tools = [next_num]  # Ensure tools are correctly referenced

  agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)
  agent_executor = AgentExecutor(
        agent=agent, # Pass the agent instance here
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3,
        max_execution_time=60
    )
  result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(user_query=user_query)}
    )

  return result["output"]



In [6]:
agent_007('what next of 0 ?')



> Entering new AgentExecutor chain...
To determine the next number after 0, I'll need to use the provided tool that adds 12 to the input number. 

Action: next_num_tool  
Action Input: 0  twelveI now know the final answer.  
Final Answer: twelve

> Finished chain.


'twelve'